In [1]:
import serial
import time 
from datetime import datetime
import os
import csv
from tkinter import Tk
from tkinter.filedialog import askdirectory
#to get port number type 'ls /dev/tty.*' in terminal

Press esc M to change cell to text (markdown)
figure out how to do keyboard interrrupt and just keep that, keyboard module seems to fuck things up. 

In [4]:
#run this cell to test behavior rig for 60 sec
port = '/dev/tty.usbmodem101'  # Replace with your Arduino's port
baudrate = 115200  # make sure this matches the baud rate in the Arduino sketch (.ino file)
#request user for identiying information
#in the future commands sent to arduino can be changed based on the training stage 
animal_ID = input("Enter Animal ID: ")
training_stage = input("Enter training stage: ")

#initialize serial connection
ser = serial.Serial(port, baudrate, timeout=1)
time.sleep(2)  # wait for the connection to be established
#TRIGGER CAMERA, 2P HERE
#initialize list for sensor data storage 
sensor_data = []

#send start command to Arduino
start_time = time.time() #use for duration based arduino script start/stop
ser.write(b'S')
behav_active = True
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
print(f'{timestamp} - Sent start command to Arduino.')

try:
    # Read data from Arduino
    while behav_active:
        if ser.in_waiting > 0:
            try:
                line = ser.readline().decode('utf-8', errors='ignore').rstrip()
                timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
                print(f'{timestamp} - Arduino: {line}')
                sensor_data.append([timestamp, line])
            except UnicodeDecodeError:
                print("Failed to decode serial data")
                continue

        # Check if 20 seconds have passed to send the stop command
        if time.time() - start_time >= 20:
            ser.write(b'X')
            print('Sent stop command to Arduino.')
            behav_active = False

except KeyboardInterrupt:
    ser.write(b'X')
    print("Interrupted by user. Sent stop command to Arduino.")
finally:
    # Open a dialog to choose a directory to save the CSV file
    root = Tk()
    root.withdraw()  # Hide the main tkinter window
    save_directory = askdirectory(title="Select Directory to Save CSV File")

    if save_directory:  # Check if the user selected a directory
        csv_filename = f"{animal_ID}_{training_stage}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        csv_filepath = os.path.join(save_directory, csv_filename)

        # Save sensor data to CSV
        with open(csv_filepath, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Timestamp", "Sensor Data"])
            writer.writerows(sensor_data)
        print(f"Sensor data saved to {csv_filepath}")
    else:
        print("No directory selected. Data not saved.")
    #close serial connection
    ser.close()

Enter Animal ID:  test
Enter training stage:  lick


2024-08-13 18:45:23.920 - Sent start command to Arduino.
18:45:23.928 - Arduino: Arduino doing things, Arduino started
18:45:23.936 - Arduino: bar: 425
18:45:23.936 - Arduino: lick: 2
18:45:23.940 - Arduino: Solenoid Activated
18:45:23.940 - Arduino: 1
18:45:23.957 - Arduino: bar: 486
18:45:23.957 - Arduino: lick: 3
18:45:23.957 - Arduino: 1
18:45:23.977 - Arduino: bar: 520
18:45:23.977 - Arduino: lick: 2
18:45:23.977 - Arduino: 1
18:45:23.998 - Arduino: bar: 559
18:45:23.998 - Arduino: lick: 2
18:45:23.998 - Arduino: 0
18:45:24.018 - Arduino: bar: 602
18:45:24.018 - Arduino: lick: 3
18:45:24.018 - Arduino: 0
18:45:24.039 - Arduino: bar: 619
18:45:24.043 - Arduino: lick: 3
18:45:24.043 - Arduino: 0
18:45:24.063 - Arduino: bar: 619
18:45:24.063 - Arduino: lick: 2
18:45:24.063 - Arduino: 0
18:45:24.084 - Arduino: bar: 635
18:45:24.084 - Arduino: lick: 2
18:45:24.084 - Arduino: 0
18:45:24.108 - Arduino: bar: 668
18:45:24.108 - Arduino: lick: 2
18:45:24.108 - Arduino: 0
18:45:24.129 - Ardu

I have decided that keeping code inside a main script is totally irrelvant in jupyter notebook
so I will instead just put the behavior triggering code within the cell directly. 

In [15]:
def main():
    animal_id = input("Enter Animal ID: ")
    session_type = input("Enter Session Type: ")

if __name__ == '__main__':
    main()

Enter Animal ID:  animal1
Enter Session Type:  lick_direct


In [16]:
animal_id = input("Enter Animal ID: ")
session_type = input("Enter Session Type: ")

Enter Animal ID:  animal1
Enter Session Type:  lick


In [4]:
#old version, keep for history
def main():
    port = '/dev/tty.usbmodem101'  # Replace with your Arduino's port
    baudrate = 115200
    solenoidCount = 0
    # Initialize serial connection
    ser = serial.Serial(port, baudrate, timeout=1)
    time.sleep(2)  # Wait for the connection to be established

    # Send start command to Arduino
    ser.write(b'S')
    print('Sent start command to Arduino.')

    try:
        # Read data from Arduino
        while True:
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').rstrip()
                print(f'Arduino: {line}')
                if line == 'Solenoid Activated':
                    solenoidCount+=1
                if solenoidCount >10:
                    break  # Exit loop after solenoid activation (modify as needed)

            # Check for stop condition or other conditions as needed
            # e.g., send stop command 'X' to Arduino
            # ser.write(b'X')
            # break
    except KeyboardInterrupt:
        print("Interrupted by user")
        ser.write(b'X')  # Send stop command to Arduino
    finally:
        # Close serial connection
        ser.close()

# Ensure the main function is called if this script is executed directly
if __name__ == '__main__':
    main()


Sent start command to Arduino.
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 1
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 1
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 1
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: 